In [1]:
import os
import pandas as pd
import glob
import math
import emcee
import string
import corner
import numpy as np
from tqdm import tqdm
from numba import jit
import concurrent.futures
from multiprocessing import Pool
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
from scipy.integrate import odeint
import pdse_project as pdse
import seaborn as sns
import scipy.stats as stats
import sys

# Find and sort all text files in the ./data directory
data_files = sorted(glob.glob("./data/*.txt"))

# Dictionary to store the parsed data, grouped by data type
full_data = {}

# Loop through each file in the data_files list
for file in data_files:
    # Extract the data type (treatment and cell type) from the file name
    data_type = os.path.basename(file).split('/')[-1].split('_c')[0]
    
    # Extract the mouse name (cohort identifier) from the file name
    mouse_name = 'c' + file.split('/')[-1].split('_c')[-1].split('_t')[0]
    
    # Extract the treatment days from the file name (as integers)
    t_days = np.array([int(t) for t in file.split('/')[-1].split('_c')[-1].split('.txt')[0].split('_t')[1:]])
    
    # Load the measurement data from the file (time, tumor volume)
    data = np.loadtxt(file)
    
    # Extract the specific treatment from the data type (e.g., radiation or control)
    treatment = os.path.basename(data_type).split('_')[0]
    
    # If this data type hasn't been seen before, create a new list in full_data
    if data_type not in full_data:
        full_data[data_type] = []
    
    # Append the mouse's data (including name, tumor measurements, treatment, and treatment days) to the appropriate group
    full_data[data_type].append({
        'name': mouse_name,
        'data': data,
        'treatment': treatment,
        'treatment_days': t_days
    })

all_models = []
for group in ['io_sensitive']:
    all_mice = []
    # Adiciona o nome de cada rato à lista all_mice
    for mouse_data in full_data[group]:
        all_mice.append(mouse_data['name'])  # Corrigido: usar append sem atribuição   

    for mouse in all_mice:
        # print(f"Processando o rato {mouse} no grupo {group}...")  # Adicionado para diagnóstico

        # Localizar o arquivo .npz usando o mouse_name
        files_location = f'./Output_Calibration/ll_pars_{group}_{mouse}_*.npz'
        all_chains = sorted(glob.glob(files_location))
        
        # Verificar se há arquivos .npz encontrados
        if len(all_chains) == 0:
            print(f"Não foram encontrados arquivos .npz para o rato {mouse} no grupo {group}")
            continue

        # Encontrar o mouse no 'full_data' com base no nome do rato
        mouse_data = next((m for m in full_data[group] if m['name'] == mouse), None)
        if mouse_data is None:
            print(f"Mouse {mouse} não encontrado no grupo {group}")
            continue

        # Acessar os dados de volume tumoral diretamente pela chave 'data'
        n = len(mouse_data['data'])  # número de observações

        text_to_print = mouse + ' '
        model_to_print = 'Mouse '

        # Iterar sobre os arquivos de parâmetros encontrados
        for chain in all_chains:
            # print(f"Processando o arquivo {chain} para o rato {mouse}...")  # Adicionado para diagnóstico
            model_name = chain.split('_')[-1].split('.')[0]
            # print(model_name)
            # if model_name not in all_models:
            #     # print(f"Modelo {model_name} não está na lista permitida. Pulando...")
            #     continue  # Pula este modelo se ele não estiver em all_models

            npzfile = np.load(chain)
            max_ll = npzfile['max_ll']
            theta = npzfile['pars']
            k = len(theta)
            
            # Cálculo da BIC (Bayesian Information Criterion)
            bic_value = k * np.log(n) - 2 * max_ll
            text_to_print += f"& {bic_value:.2f} "
            
            # Extrair o nome do modelo do nome do arquivo .npz
            model_name = chain.split('_')[-1].split('.')[0]
            model_to_print += f"& {model_name} "

        # Imprimir os resultados formatados
        print(model_to_print + '\\\ \hline')
        print(text_to_print + '\\\ \hline')


<>:111: SyntaxWarning: invalid escape sequence '\ '
<>:112: SyntaxWarning: invalid escape sequence '\ '
<>:111: SyntaxWarning: invalid escape sequence '\ '
<>:112: SyntaxWarning: invalid escape sequence '\ '
C:\Users\guilh\AppData\Local\Temp\ipykernel_21168\3327644409.py:111: SyntaxWarning: invalid escape sequence '\ '
  print(model_to_print + '\\\ \hline')
C:\Users\guilh\AppData\Local\Temp\ipykernel_21168\3327644409.py:112: SyntaxWarning: invalid escape sequence '\ '
  print(text_to_print + '\\\ \hline')


Mouse & AccelCD & AccelED & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & EDwithExpFactor & EDwithLogFactor & EDwithSinFactor & ExponentialDecay & MichaelisMenten & dePillisRadunskayaLaw \\ \hline
c0_m28 & 240.30 & 239.10 & 241.95 & 241.96 & 241.96 & 239.34 & 241.95 & 241.96 & 241.95 & 239.09 & 239.01 & 241.96 \\ \hline
Mouse & AccelCD & AccelED & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & EDwithExpFactor & EDwithLogFactor & EDwithSinFactor & ExponentialDecay & MichaelisMenten & dePillisRadunskayaLaw \\ \hline
c0_m35 & 313.28 & 318.44 & 315.50 & 316.44 & 315.65 & 313.32 & 317.19 & 317.63 & 316.83 & 318.13 & 342.16 & 330.74 \\ \hline
Mouse & AccelCD & AccelED & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & EDwithExpFactor & EDwithLogFactor & EDwithSinFactor & ExponentialDecay & MichaelisMenten & dePillisRadunskayaLaw \\ \hline
c0_m3 & 188.79 & 183.18 & 184.67 & 184.68 & 184.67 & 184.15 & 184.68 & 184.68 & 184.